# EnigmaNet: A Neural Network Framework for Eplipesy Classification
### This is a script that uses Keras to predict epilepsy outcomes

## Script Details
This script is designed to run with the following configuration:
* Keras
* TensorFlow backend
* PlaidML backend (if using Radeon GPU)
* Tabular data (Excel or CSVs)

## Dependencies
Run the following command to pip install all dependencies
```
pip install numpy pandas sklearn matplotlib scipy keras_tqdm
```

Also install NeuroCombat at https://github.com/ncullen93/neuroCombat

## Path to Data
Define path to tabular data


In [162]:
csvPath = '/Users/sid/Documents/Projects/Enigma-ML/Dataset/T1/all.csv'

## Define Script Parameters
Define parameters that control script flow

In [170]:
classCol = 'Dx'             # Class labels
siteCol = 'Site'            # Site or scanner column name
dBegin = 'ICV'              # Column where data begins
dEnd = 'R_insula_surfavg'   # Column where data ends
cBegin = 'Site'             # Column where covariates/demographics begin
cEnd = 'Sex'                # Column where covariates/demographics end
fillmissing = True          # Fill missing?
harmonize = True            # Run ComBat harmonization?
scaleData = True            # Rescale data?
dataSplit = 0.50            # Percent of data to remove for validation
nEpochs = 250               # Training number of epochs
bSize = 30                  # Training batch size
plotType = 'Normal'         # Type of ComBat graphs to save ('Histogram' or 'Normal')
hyperOpt = True             # Optimize hyperparameters with Talos?
talosName = 'Enigma_T1'     # Talos dataset name
talosN = '2'                # Talos experiment number

## Define ComBat Harmonization Parameters
Python implementation of ComBat is currently being used, which can be found at https://github.com/ncullen93/neuroCombat

In [163]:
# Combat Variables
if harmonize:
    batchVar = 'Site'           # Batch effect variable
    discreteVar = ['Dx','Sex']  # Variables which are categorical that you want to predict
    continuousVar = ['Age']     # Variables which are continuous that you want to predict

## Import Required Modules
Import all modules required to process the script

In [164]:
# SKLearn items
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import sklearn.metrics as skm
# Keras items
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.activations import relu, softmax
from keras.optimizers import Adam, Nadam
from keras.callbacks import EarlyStopping
from keras_tqdm import TQDMNotebookCallback
import talos as ta
from talos.metrics.keras_metrics import fmeasure_acc
# Other items
import numpy as np
import pandas as pd
from neuroCombat import neuroCombat
import matplotlib.pyplot as plt
import scipy.stats
import random
import os

## Dependent Function
Functions that the scrips will depend on

In [165]:
def classfill(dFrame, classCol, siteCol, idxRange):
    """Fills missing values with means of a class
    
    Inputs
    ------
    dFrame:   Pandas dataframe to process (type: dataframe)
            
    classCol: String indicating dataframe column name containing class information

    siteCol:  String indicating dataframc column name containing class information
    
    idxRange: 2x1 vector indicating lower and upper bound of data to fill in dataframe
              idxRange[0] is lower bound
              idxRange[1] is upper bound

    Returns
    -------
    data:     Dataframe will all missing values filled
    """
    uniqClass = dFrame[classCol].unique()                   # All unique classes
    uniqSites = dFrame[siteCol].unique()                    # All unique sites
    print('...found ' + str(uniqClass.size) + ' classes across ' + str(uniqSites.size) + ' sites')
    print('...filling missing data with class means')
    data = dFrame.loc[:, idxRange[0]:idxRange[1]]           # Extract all numerical value from 'dBegin' onwards
    for site in uniqSites:
        siteIdx = dFrame.loc[:, siteCol] == site            # Index where site is uniqSite = site
        for cls in uniqClass:
            classIdx = dFrame.loc[:, classCol] == cls       # Index where class is uniqClass = cls
            idx = siteIdx & classIdx                        # Index where both class and site indexes are true
            for col in range(len(data.columns)):            # Iterate along each column
                nanIdx = data.iloc[: ,col].isnull()         # Index where NaNs occur per feature
                nanIdx_i = nanIdx & idx                     # Index where NaNs occur per feauture, per site, per class
                if np.sum(nanIdx_i) > 0:
                    mean = np.nanmean(data.iloc[:, col][idx]) # Compute mean of non-NaNs# If there are any Nans...
                    data.iloc[:, col][nanIdx_i] = mean      # Replace NaNs with mean
    dFrame.loc[:, idxRange[0]:idxRange[1]] = data           # Substitute dataframe with corrected data
    return dFrame

def minorityclass(dFrame, classCol):
    """Returns the minority class label in a set of binary class labels
    Inputs
    ------
    dFrame: Pandas Dataframe containing tabular data

    classCol: Column label containing class information (string)

    Returns
    -------
    minorClass: String depicting minor class

    disparity: Integer indicating disparity between minor and major classes
    """
    uniqClass = dFrame[classCol].unique()  # All unique classes
    nClass = np.zeros((uniqClass.shape), dtype=int)
    for i, classVal in enumerate(uniqClass):
        nClass[i] = np.sum(dFrame.loc[:, classCol] == classVal)
    minorIdx = np.argmin(nClass)
    minorClass = uniqClass[minorIdx]
    majorIdx = np.argmax(nClass)
    disparity = nClass[majorIdx] - nClass[minorIdx]
    return minorClass, disparity

## Load Files and Manipulate
Load tabular data and prime it for deep learning
### Load Data and Perform Sanity Check
Load CSV into a Pandas Dataframe, then obtain all unique classes to ensure only relevant classes are present.

In [166]:
dFrame = pd.read_csv(csvPath)           # Create Dataframe

# Integrity check
print('Found classes: ' + str(dFrame.loc[:, 'Dx'].unique()))

Found classes: [1 0]


### Manipulate Data
This section fills in missing values and performs ComBat harmonization across multiple sites. It is necessary to fill in missing data prior to harminization as a prerequisity to running ComBat.

In [167]:
# Fill missing
if fillmissing:
    dFrame = classfill(dFrame, 'SDx', siteCol, [dBegin, dEnd])
else:
    print('...skip fill missing')

# Run combat
if harmonize:
    cData = neuroCombat(data=dFrame.loc[:,dBegin:dEnd],
                          covars=dFrame.loc[:,cBegin:cEnd],
                          batch_col=batchVar,
                          discrete_cols=discreteVar,
                          continuous_cols=continuousVar)

data = np.array(dFrame.loc[:, dBegin:dEnd])     # Preserve non-harmonized data

# Scale data
scaler = StandardScaler()   # Initialize scaler
if scaleData:
    if harmonize:
        cData = scaler.fit_transform(cData)
    data = scaler.fit_transform(data)

...found 2 classes across 16 sites
...filling missing data with class means
Creating design matrix..
Standardizing data across features..
Fitting L/S model and finding priors..
Finding parametric adjustments..
Final adjustment of data..


## Split Data for Training and Evaluation
Data needs to be split for trianing and evaluation

In [168]:
# Split into training and validation sets and scale
if harmonize:
    x_train, x_test, y_train, y_test = train_test_split(cData, dFrame.loc[:, classCol],
                                                        test_size=dataSplit,
                                                        random_state=42,
                                                        stratify=dFrame.loc[:, classCol])
else:
    x_train, x_test, y_train, y_test = train_test_split(data, dFrame.loc[:, classCol],
                                                        test_size=dataSplit,
                                                        random_state=42,
                                                        stratify=dFrame.loc[:, classCol])
# Oversample minority class suing SMOTE
x_train, y_train = SMOTE().fit_resample(x_train, y_train)

## Build Model and Train
A perceptron is constructed here for training

In [169]:
def EnigmaNet(x_train, y_train, x_test, y_test, params):
    """Constructs Sequential NN model based on Talos optimization parameters

    Inputs
    ------
    x_train: Training dataset
    y_train: Training class labels
    x_test:  Validation dataset
    y_test:  Validation class labels
    params:  Talos gridsearch parameters
    
    Returns
    -------
    model:   Sequential model based on input parameters
    history: Fitted model based on training set
    """
    # Initialising the ANN
    model = Sequential()

    # Add initial layer
    model.add(Dense(params['first_neuron'], input_dim=x_train.shape[1],
                    activation=params['activation'],
                    kernel_initializer=params['kernel_initializer']))

    # Adding dropout to prevent overfitting
    model.add(Dropout(params['dropout']))

    # Adding hidden layers
    model.add(Dense(1, activation=params['last_activation'],
                    kernel_initializer=params['kernel_initializer']))

    # Adding the output layer
    model.add(Dense(output_dim=1, init='uniform', activation='sigmoid'))

    # criterion loss and optimizer
    model.compile(loss=params['losses'],
                  optimizer=params['optimizer'](),
                  metrics=['acc', fmeasure_acc])

    # Fitting the ANN to the Training set
    early_stopping = EarlyStopping(monitor='val_loss', patience=2)
    history = model.fit(x_train, y_train,
                        validation_data=[x_test, y_test],
                        batch_size=params['batch_size'],
                        epochs=params['epochs'],
                        verbose=False,
                        callbacks=[TQDMNotebookCallback(leave_inner=False, leave_outer=True)])
    return history, model

## Construct Talos Parameters
Here, instructions will be constructed for Talos to optimize hyperparameters for EnigmaNet. The call to run Talos will also be initiated here.

In [171]:
if hyperOpt:
    # Construct hyperparameter optimization paramaeters
    p = {'first_neuron':[64, 128, 256],
         'hidden_layers':[0, 1, 2],
         'batch_size': [30 , 40 , 50],
         'epochs': [200, 1000],
         'dropout': [0, 0.2, 0.5, 1],
         'kernel_initializer': ['uniform', 'normal'],
         'optimizer': [Nadam, Adam],
         'losses': ['binary_crossentropy'],
         'activation':[relu, softmax],
         'last_activation': ['sigmoid', 'softmax']}


    # Run Talos with parameters
    scan_object = ta.Scan(x=x_train,
                          y=y_train,
                          model=EnigmaNet,
                          params=p,
                          dataset_name=talosName,
                          experiment_no=talosN,
                          grid_downsample=0.005)
else:
        # Construct hyper parameters from a prior optimization
        from keras.activations import relu
        from keras.losses import binary_crossentropy
        from keras.optimizers import Adam
        p = {'first_neuron':[64],
         'hidden_layers':[2],
         'batch_size': [30],
         'epochs': [1000],
         'dropout': [0.2],
         'kernel_initializer': ['normal'],
         'optimizer': [Adam],
         'losses': [binary_crossentropy],
         'activation':[relu],
         'last_activation': ['sigmoid']}7








  0%|          | 0/17 [00:00<?, ?it/s]








  6%|▌         | 1/17 [00:13<03:42, 13.91s/it]








 12%|█▏        | 2/17 [00:28<03:33, 14.23s/it]








 18%|█▊        | 3/17 [01:28<06:30, 27.86s/it]








 24%|██▎       | 4/17 [02:14<07:12, 33.24s/it]

...








 76%|███████▋  | 13/17 [05:26<01:31, 22.99s/it]








 82%|████████▏ | 14/17 [06:11<01:29, 29.69s/it]








 88%|████████▊ | 15/17 [06:54<01:07, 33.72s/it]








 94%|█████████▍| 16/17 [07:05<00:26, 26.91s/it]








100%|██████████| 17/17 [07:49<00:00, 32.00s/it]

## Access Model Optimization Results
Now that the model's hyper parameters have been optimized, access them with the following portion.

## Evaluation of Model
Evaluation based on holdout data. A confusion matrix is printed out

In [ ]:
# Load GridSearch report
# modelRep = ta.Reporting(talosName + '_' + talosN + '.csv')


# Find best model based of validation accuracy
GridFrame = modelRep.data.head(-3)
bestParams = modelRep.best_params('val_acc')
idxBest = modelRep.rounds2high('val_acc')
modelBest = GridFrame.loc[idxBest, :]
print(modelBest)

newModel = EnigmaNet(x_train, y_train, x_test, y_test, bestParams)

In [ ]:
# Predicting the Test set results
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5)
# Making the Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
accuracy = skm.accuracy_score(y_test, y_pred)
precision = skm.precision_score(y_test, y_pred)
recall = skm.recall_score(y_test, y_pred)
kappa = skm.cohen_kappa_score(y_test, y_pred)
resultStr = '''----------Validation Results----------
Confusion Matrix: 
{0}
    Accuracy:  {1:0.2f}%
    Precision: {2:0.2f}%
    Recall:    {3:0.2f}%
    Kappa:     {4:0.2f}%
--------------------------------------
'''.format(np.array(cm), accuracy*100, precision*100, recall*100, kappa*100)
print(resultStr)

## Create Plots
Create and save plots in work directory

In [ ]:
# Form Graph Path
pwd = os.getcwd()
savePathModel = os.path.join(pwd, 'model_fit.png')
savePathComBat = os.path.join(pwd, 'combat.png')

# Plot training & validation accuracy values
with plt.style.context('ggplot'):
    plt.subplot(121)
    plt.plot(history.history['acc'])
    # plt.plot(history.history['val_acc'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='lower right')

    # Plot training & validation loss values
    plt.subplot(122)
    plt.plot(history.history['loss'])
    # plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper right')
plt.savefig(savePathModel, dpi=600)

# Plot ComBat before & after
if harmonize:
    szSubPlot = 4                                                                   # Number of features to plot
    nBins = 20                                                                      # Number of bins

    uniqSites = dFrame.loc[:,'Site'].unique()
    with plt.style.context('ggplot'):                                               # Plotting style
        fig, axs = plt.subplots(np.sqrt(szSubPlot).astype(int), np.sqrt(szSubPlot).astype(int))
        for axsNum, axsIdx in enumerate(axs.reshape(-1)):                                              # Iterate over subplots
            plotIdx = random.randint(0,len(dFrame.loc[:,dBegin:dEnd].columns))      # Index random headers
            for s in uniqSites:
                siteIdx = dFrame.loc[:, 'Site'] == s
                nBefore, bBefore = np.histogram(data[siteIdx.values, plotIdx],      # Bin count before
                                               bins=nBins,
                                               density=True)
                nAfter, bAfter = np.histogram(cData[siteIdx.values, plotIdx],       # Bin count after
                                             bins=nBins,
                                             density=True)

                mBefore = np.zeros((nBins,))
                mAfter = np.zeros((nBins,))
                for i  in range(len(bBefore)-1):                                    # Get median of bin edges
                    mBefore[i] = np.median([bBefore[i], bBefore[i + 1]])            # Median of bin edges (before)
                    mAfter[i] = np.median([bAfter[i], bAfter[i + 1]])               # Median of bin edges (after)

                siteIdx = dFrame.loc[:,'Site'] == s                                 # Extract data for a site
                muBefore = np.mean(data[siteIdx.values, plotIdx])
                muAfter = np.mean(cData[siteIdx.values, plotIdx])
                stdBefore = np.std(data[siteIdx.values, plotIdx])
                stdAfter = np.std(cData[siteIdx.values, plotIdx])
                yBefore = scipy.stats.norm.pdf(mBefore, muBefore, stdBefore)
                yAfter = scipy.stats.norm.pdf(mAfter, muAfter, stdAfter)
                if plotType == 'Histogram':
                    yBefore = nBefore
                    yAfter = nAfter
                elif plotType == 'Normal':
                    yBefore = scipy.stats.norm.pdf(mBefore, muBefore, stdBefore)
                    yAfter = scipy.stats.norm.pdf(mAfter, muAfter, stdAfter)

                axsIdx.plot(mBefore, yBefore,                                       # Plot on subplot(axsIdx) before
                                  color='#3a4750',
                                  alpha=0.25)

                axsIdx.plot(mAfter, yAfter,                                         # Plot on subplot(axsIdx) after
                                  color='#d72323',
                                  alpha=0.25)

                if axsNum == 0 or axsNum == 2:
                    axsIdx.set_ylabel('NORMALIZED SUBJECTS',
                                      fontsize=6)

                axsIdx.set_xlabel(dFrame.loc[:, dBegin:dEnd].columns[plotIdx].upper(),
                                  fontsize=6)

        fig.legend(['Before ComBat', 'After ComBat'],                               # Legend
                   loc = 'lower right',
                   ncol=2,
                   fancybox=True,
                   bbox_to_anchor=(0.5,-0.1))
        plt.subplots_adjust(wspace=0.2, hspace=0.5)

    plt.tight_layout()
    plt.savefig(savePathComBat, dpi=600)